In [30]:
import pandas as pd
from random import randint
import math
from node import Node
from edge import Edge
from cluster import Cluster

In [50]:
class DotMake:
    def __init__(self):
        self.nodes = dict()
        self.clusters = []
        self.clusterPair = dict()
        self.edges = dict()
        self.clusterCount = 1
    
    def addNode(self, val, color= None, shape = None, clusterName = None):
        if val in self.nodes:
            raise Exception("node already done")
        tmp = Node(val, color, shape)
        
        self.nodes[val] = tmp
        if clusterName !=None:
            try:
                position = self.clusters.index(clusterName)
                print(type(self.clusterPair[clusterName]))
                self.clusterPair[clusterName].addNodes(self.nodes[val])
            except ValueError:
                clusterC = "cluster" + str(self.clusterCount)
                self.clusterCount+=1
                newCluster = Cluster(clusterC, clusterName)
                newCluster.addNodes(self.nodes[val])
                self.clusters.append(clusterName)
                self.clusterPair[clusterName] = newCluster
        return tmp
    
    def addEdge(self, start:Node, end:Node,  color = None, ltail = None, lhead = None):
        tmp = Edge(start,end,color,ltail,lhead)
        if tmp.label in self.edges:
            raise Exception("edge already done")
        self.edges[tmp.label] = tmp
    
    def addCluster(self, name):
        if name in self.clusterPair:
            raise Exception("cluster already made")
        clusterC = "cluster" + str(self.clusterCount)
        self.clusterCount+=1
        newCluster = Cluster(clusterC, name)
        self.clusters.append(name)
        self.clusterPair[name] = newCluster


    
    def __str__(self):
        total = "digraph G {\ncompound=true;\n"
        nodesSeen = set()
        for indCluster in self.clusters:
            total = total + f"subgraph {self.clusterPair[indCluster].label} {{\nlabel=\"{self.clusterPair[indCluster].name}\";\n"
            for indNode in self.clusterPair[indCluster].nodes:
                total = total + str(indNode) + '\n'
                nodesSeen.add(indNode.label)
            total = total + '}\n'
        for indNode in self.nodes:
            if indNode in nodesSeen:
                continue
            total = total + str(indNode) + '\n'
        for indEdge in self.edges:
            total = total + str(indEdge) + '\n'
        total = total + '}'
        return total
            
    


In [51]:
#generate the graph
def generateGraph(nodes, linking):
    graph = dict()
    for i in range(nodes):
        graph[i] = []
    for i in range(nodes):
        for j in range(nodes):
            if i == j:
                continue
            prob = randint(1,int(linking.as_integer_ratio()[1]))
            if(prob <= int(linking.as_integer_ratio()[0])):
                graph[i].append(j)
    return graph
smallGraph = generateGraph(20,.1)
totalDegree = 0
graph = DotMake()
gvComp = dict()
even = graph.addCluster("Even")
odd = graph.addCluster("Odd")
for startN in smallGraph:
    if startN not in gvComp:
        if startN%2 == 0:
            gvComp[startN] = graph.addNode(val = str(startN))
        else:
            gvComp[startN] = graph.addNode(val = str(startN))
    for endN in smallGraph[startN]:
        if endN %2 == startN %2:
            continue
        if endN not in gvComp:
            if endN %2 == 0:
                gvComp[endN] = graph.addNode(val = str(endN))
            else:
                gvComp[endN] = graph.addNode(val = str(endN))
        graph.addEdge(start = gvComp[startN],end = gvComp[endN])
